# Instalamos e importamos librerías

In [ ]:
!pip install pyarrow
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubun

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Autenticamos con Google Drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Creamos el Spark Context

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

NameError: ignored

#Ejercicio 34

In [ ]:
downloaded = drive.CreateFile({'id':"1ii6j8Mex52FFWCLa4coVX6s47GuXN0B3"})
downloaded.GetContentFile('logs.csv') 

In [ ]:
texts = pd.read_csv('logs.csv')
texts.to_parquet('logs.parquet')
del texts
df = sqlContext.read.parquet('logs.parquet')
logs_rdd = df.rdd

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
logs_rdd.take(5)

[Row(item_id=1, timestamp='2004-12-23T07:10:09Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: 'Si aprendés Pensamiento Científico ... TE VA S AENTERAR'", logtype='delete', action='delete', title='Verificacionismo'),
 Row(item_id=2, timestamp='2004-12-23T07:14:09Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: '{{destruir}}en algun lugar lejano pero tan cerca como el largo del cuello de una jirafa se encontraba una torre'", logtype='delete', action='delete', title='Rapunzel'),
 Row(item_id=3, timestamp='2004-12-23T07:26:10Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: '#REDIRECT [[Flash]]'", logtype='delete', action='delete', title='Macromedia Flash'),
 Row(item_id=4, timestamp='2004-12-23T07:55:09Z', contributor_username='Sanbec', contributor_id=7856.0, contributor_ip=None, comment='Sin información de origen ni de 

In [ ]:
users_blocks = logs_rdd.filter(lambda x: x.action != None and 'block' in x.action).map(lambda x: (x.contributor_username, 1)).cache()

In [ ]:
users_blocks.take(5)

[('Dodo', 1), ('Dodo', 1), ('Dodo', 1), ('Dodo', 1), ('Comae', 1)]

In [ ]:
users_blocks.reduceByKey(lambda x,y: x+y).reduce(lambda x,y: x if x[1]>y[1] else y)

('Magister Mathematicae', 30155)

#Ejercicio 18


In [ ]:
downloaded = drive.CreateFile({'id':"1caGHf_GS2-XSVxggt4WnFSGymgsy3DHq"})
downloaded.GetContentFile('languages.csv') 

In [ ]:
texts = pd.read_csv('languages.csv')
texts.to_parquet('languages.parquet')
del texts
df = sqlContext.read.parquet('languages.parquet')
languages_rdd = df.rdd

In [ ]:
comprension = languages_rdd.filter(lambda x: x.babel_level in ('1', '2', '3', '4', 'N', '5'))
basico = languages_rdd.filter(lambda x: x.babel_level in ('2', '3', '4', 'N', '5'))
avanzado = languages_rdd.filter(lambda x: x.babel_level == '5')

In [ ]:
categorias = []

for rdd in (comprension, basico, avanzado):

  categorias.append(rdd.filter(lambda x: x.babel_lang != None).map(lambda x: (x.babel_lang.lower(), 1)).reduceByKey(lambda x,y: x+y))

Comprenden el idioma

In [ ]:
categorias[0].collect()

[('fr', 3488),
 ('ab', 1),
 ('acf', 1),
 ('af', 10),
 ('agr', 1),
 ('akk', 1),
 ('aln', 2),
 ('am', 2),
 ('an', 57),
 ('ang', 6),
 ('ar', 161),
 ('arc', 3),
 ('arn', 15),
 ('ary', 1),
 ('ase', 2),
 ('ast', 152),
 ('avk', 1),
 ('ay', 4),
 ('az', 6),
 ('azb', 1),
 ('bar', 6),
 ('bcl', 1),
 ('be', 24),
 ('bew', 1),
 ('bg', 30),
 ('bjn', 1),
 ('bn', 1),
 ('br', 2),
 ('bs', 13),
 ('ca', 1289),
 ('cak', 1),
 ('cas', 1),
 ('cbk', 1),
 ('ceb', 3),
 ('ch', 2),
 ('chr', 1),
 ('ckb', 1),
 ('cmn', 1),
 ('cnt', 1),
 ('co', 7),
 ('cs', 44),
 ('csb', 1),
 ('csc', 1),
 ('csn', 1),
 ('cu', 1),
 ('cv', 2),
 ('cy', 7),
 ('da', 41),
 ('de', 1644),
 ('de-at', 2),
 ('egy', 1),
 ('el', 108),
 ('eml', 1),
 ('en', 9454),
 ('en-ca', 2),
 ('en-gb', 8),
 ('eo', 258),
 ('es', 10223),
 ('et', 15),
 ('eu', 229),
 ('ext', 19),
 ('fa', 30),
 ('fax', 1),
 ('fi', 52),
 ('fil', 1),
 ('fo', 2),
 ('frp', 6),
 ('fur', 2),
 ('fy', 2),
 ('ga', 19),
 ('gan', 1),
 ('gd', 2),
 ('gl', 472),
 ('gn', 47),
 ('gom-latn', 1),
 ('grc',

Nivel de escritura y lectura de basico para arriba


In [ ]:
categorias[1].collect()

[('fr', 2016),
 ('af', 4),
 ('aln', 2),
 ('am', 2),
 ('an', 19),
 ('ang', 3),
 ('ar', 69),
 ('arc', 2),
 ('arn', 5),
 ('ary', 1),
 ('ase', 1),
 ('ast', 94),
 ('avk', 1),
 ('az', 5),
 ('azb', 1),
 ('bar', 4),
 ('be', 16),
 ('bew', 1),
 ('bg', 15),
 ('bn', 1),
 ('br', 2),
 ('bs', 9),
 ('ca', 1029),
 ('cas', 1),
 ('ch', 2),
 ('chr', 1),
 ('ckb', 1),
 ('cmn', 1),
 ('cnt', 1),
 ('co', 3),
 ('cs', 27),
 ('csn', 1),
 ('cv', 1),
 ('cy', 2),
 ('da', 23),
 ('de', 913),
 ('el', 45),
 ('eml', 1),
 ('en', 8629),
 ('en-ca', 1),
 ('en-gb', 8),
 ('eo', 116),
 ('es', 9862),
 ('et', 6),
 ('eu', 173),
 ('ext', 10),
 ('fa', 23),
 ('fax', 1),
 ('fi', 30),
 ('fil', 1),
 ('fo', 2),
 ('frp', 3),
 ('fur', 1),
 ('ga', 7),
 ('gan', 1),
 ('gl', 354),
 ('gn', 35),
 ('gom-latn', 1),
 ('grc', 25),
 ('gsw', 9),
 ('guc', 3),
 ('gym', 1),
 ('haw', 1),
 ('he', 50),
 ('hi', 6),
 ('hr', 25),
 ('hu', 22),
 ('hy', 4),
 ('ia', 10),
 ('id', 13),
 ('ie', 2),
 ('io', 3),
 ('is', 4),
 ('ise', 1),
 ('it', 903),
 ('ja', 165),
 ('j

Uso avanzado del idioma


In [ ]:
categorias[2].collect()

[('ast', 2),
 ('bs', 1),
 ('ca', 14),
 ('de', 24),
 ('el', 3),
 ('en', 315),
 ('es', 176),
 ('fa', 1),
 ('fr', 41),
 ('gl', 3),
 ('gn', 1),
 ('hr', 1),
 ('hu', 1),
 ('it', 16),
 ('ja', 2),
 ('la', 3),
 ('pl', 2),
 ('pt', 21),
 ('pt-br', 2),
 ('ru', 7),
 ('scn', 1),
 ('sh', 1),
 ('simple', 1),
 ('sr', 2),
 ('uk', 1),
 ('zh', 1),
 ('zh-hans', 1)]

#Ejercicio 19

In [ ]:
downloaded = drive.CreateFile({'id':"1k3iTFERn35WYA9zWpPMvzdnSjCdsyWaF"})
downloaded.GetContentFile('geotags.csv')

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('geotags.csv', header=True, inferSchema=True)
geotags_rdd = df.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
geotags_rdd.take(5)

[Row(gt_id='158041', gt_page_id=4328020, gt_globe='earth', gt_primary=0, gt_lat=41.50749969, gt_lon=-5.81750011, gt_dim=1000, gt_type=None, gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id='5844377', gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=13.73799992, gt_lon=-89.29199982, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id='5844381', gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=13.70199966, gt_lon=-89.20800018, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id='5844383', gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=13.39999962, gt_lon=-89.75, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id='5844388', gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=12.85000038, gt_lon=-88.0, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None)]

In [ ]:
planetas_menos_tierra = ('mercury', 'venus', 'mars', 'jupiter', 'neptune', 'saturn', 'uranus')

In [ ]:
geotags_rdd.filter(lambda x: x.gt_globe in planetas_menos_tierra).map(lambda x: (x.gt_globe, 1)).reduceByKey(lambda x,y: x+y).collect()

[('mars', 1438), ('mercury', 561), ('venus', 24)]

#Ejercicio 39

In [ ]:
logs_rdd.take(5)

[Row(item_id=1, timestamp='2004-12-23T07:10:09Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: 'Si aprendés Pensamiento Científico ... TE VA S AENTERAR'", logtype='delete', action='delete', title='Verificacionismo'),
 Row(item_id=2, timestamp='2004-12-23T07:14:09Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: '{{destruir}}en algun lugar lejano pero tan cerca como el largo del cuello de una jirafa se encontraba una torre'", logtype='delete', action='delete', title='Rapunzel'),
 Row(item_id=3, timestamp='2004-12-23T07:26:10Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: '#REDIRECT [[Flash]]'", logtype='delete', action='delete', title='Macromedia Flash'),
 Row(item_id=4, timestamp='2004-12-23T07:55:09Z', contributor_username='Sanbec', contributor_id=7856.0, contributor_ip=None, comment='Sin información de origen ni de 

In [ ]:
logs_sin_usuario = logs_rdd.filter(lambda x: x.contributor_ip != None).cache()
logs_sin_usuario.take(5)

[Row(item_id=5101331, timestamp='2011-10-13T17:53:19Z', contributor_username=None, contributor_id=None, contributor_ip='201.33.231.244', comment=None, logtype='newusers', action='autocreate', title='Usuario:Tvmundomarbr'),
 Row(item_id=5125642, timestamp='2011-10-23T01:30:39Z', contributor_username=None, contributor_id=None, contributor_ip='58.97.219.125', comment=None, logtype='newusers', action='autocreate', title='Usuario:Bdnasir'),
 Row(item_id=5145241, timestamp='2011-10-29T16:06:32Z', contributor_username=None, contributor_id=None, contributor_ip='101.128.157.140', comment=None, logtype='newusers', action='autocreate', title='Usuario:テストマン'),
 Row(item_id=5150600, timestamp='2011-10-31T14:12:58Z', contributor_username=None, contributor_id=None, contributor_ip='193.52.198.110', comment=None, logtype='newusers', action='autocreate', title='Usuario:Andrasolofoarisoa'),
 Row(item_id=5152099, timestamp='2011-11-01T02:26:40Z', contributor_username=None, contributor_id=None, contributor

In [ ]:
logs_sin_usuario.map(lambda x: (x.action, 1)).reduceByKey(lambda x,y: x+y).reduce(lambda x,y: x if x[1]>y[1] else y)

('create', 124119)

#Ejercicio 22

In [ ]:
languages_rdd.take(5)

[Row(babel_user=4502458, babel_lang='FR', babel_level='2'),
 Row(babel_user=5928200, babel_lang='aa', babel_level='0'),
 Row(babel_user=46918, babel_lang='ab', babel_level='0'),
 Row(babel_user=2050449, babel_lang='ab', babel_level='1'),
 Row(babel_user=4715583, babel_lang='ace', babel_level='0')]

In [ ]:
bilingues = languages_rdd.map(lambda x: (x.babel_user, 1)).reduceByKey(lambda x,y: x+y).filter(lambda x: x[1] == 2)

In [ ]:
joined = languages_rdd.join(bilingues).cache()
joined.take(5)

[(1724336, ('ang', 2)),
 (1724336, ('es', 2)),
 (1725400, ('ang', 2)),
 (1725400, ('es', 2)),
 (674716, ('ar', 2))]

In [ ]:
joined.map(lambda x: (x[1][0].lower(), 1)).reduceByKey(lambda x,y: x+y).takeOrdered(5, lambda x: -x[1])

[('es', 3054), ('en', 2944), ('fr', 56), ('ca', 54), ('pt', 22)]